# Object Detection using a YOLO V4 custom model training

*by Georgios K. Ouzounis, June 10th, 2021*

In this exercise we will experiment with object detection in still images using the YOLO V4 model customized for our data. The challenge will be to perform face mask detection after training a new model with images from a publically available dataset in www.kaggle.com
 

Before starting, please go to *Runtime->Change runtime type* and chose GPU as your hardware accelerator. This code will work only under this assumption.

In [ ]:
# import the relevant libraries
import numpy as np
import cv2 # openCV

In [ ]:
# check the opencv version
print(cv2.__version__)

In [ ]:
# if the openCV version is < 4.4.0 update to the latest otherwise skip this step
!pip install opencv-python==4.5.2.52

## Configure the DarkNet project

clone the darknet project

In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git

In [ ]:
%cd darknet

###  1st: change settings in the Makefile to enable GPU processing, CUDA and OpenCV

In [ ]:
!sed -i 's/GPU=0/GPU=1/g' Makefile

In [ ]:
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile

In [ ]:
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile

confirm changes

In [ ]:
!head Makefile

Double click the Makefile to open it on the right of this window.

Scroll down to line 20: ARCH = ...
and delete the two lines:

**-gencode arch=compute_35,code=sm_35 \**

**-gencode arch=compute_50,code=[sm_50,compute_50] \**

as CUDA no longer supports  these GPU architectures.

The new text block should be as follows:

**ARCH= -gencode arch=compute_52,code=[sm_52,compute_52] \
        -gencode arch=compute_61,code=[sm_61,compute_61]**


create a ```model``` directory in your Google Drive and store all critical files there


In [ ]:
%mkdir ../drive/MyDrive/my_model/
%cp Makefile ../drive/MyDrive/my_model/

### 2nd: Get pre-trained weights for new training and testing 

In [ ]:
%mkdir customization
%cd customization

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

In [ ]:
%cp yolov4.weights ../../drive/MyDrive/my_model/

### 3d: create a new custom network configuration file 

make a new copy of original configuration file

In [ ]:
%cp ../cfg/yolov4.cfg .

- Double click on yolov4.cfg to open it to the left of this window for editing. See [instructions](https://github.com/AlexeyAB/darknet#how-to-train-to-detect-your-custom-objects) for more details.
- change the following:
  1. line 3: a subdivision is the number of tiles each image is cut to for GPU processing. Change this from: **subdivisions=8 -> subdivisions=64**
  2. line 7: the resized image width. Change this from **width=608 -> width=416**
  3. line 8: the resized image height. Change  this from **height=608 -> height=416**
  4. line 19: max_batches is equal to classes\*2000 but not less than number of training images and not less than 6000. Change this from **max_batches = 500500 -> 4000** for our two classes.
  5. line 21: change line steps to 80% and 90% of max_batches. We use a single step for memory efficiency. Change this from **steps=400000,450000 -> steps=3200**
- change the last set of filters before each output layer:
  1. line 961, 1049, 1137: change from **filters=255 -> filters=21**. The rule is (classes + 5)x3 in the 3 [convolutional] before each [yolo] layer. Keep in mind that it only has to be the last [convolutional] before each of the [yolo] layers. 
- change the number of classes in each output layer:
  1. line 968, 1056, 1144: change from **classes=80 ->classes=2**.

- Press Ctrl+S to save the edited cfg file.

**REMARK**

The **max_batches** entry is set to 4000 based on the YOLO guidlines but this will result in approximately 10h of training. Since it was observed empirically that the best network weights are obtained before 2000 epochs, it is recommended to change the following to:

- **max_batches = 2000**
- **steps = 1600**

In [ ]:
!cat yolov4.cfg

back it up

In [ ]:
%cp yolov4.cfg ../../drive/MyDrive/my_model/

## Get the data

### 1st: Create the directories

In [ ]:
%cd ../
%mkdir custom_data
%cd custom_data
%mkdir images
%mkdir labels

### 2nd: create or login to your kaggle account ([www.kaggle.com](www.kaggle.com)).  

- go to https://www.kaggle.com/saurah403/face-mask-detectionimages-with-yolo-format/download and click download. Store the compressed file in your local drive and then upload it into your Google Drive. In the example below modify the path accordingly.

In [ ]:
%cd ./images/
%cp /content/drive/MyDrive/object_detection/data/archive.zip .
!unzip archive.zip -d .
%rm archive.zip

In [ ]:
%mv ./images/* . 
%rm -r images/

Check the image formats and convert all non jpg images to JPG format if necessary, and remove the originals

In [ ]:
!find . -type f | awk -F. '!a[$NF]++{print $NF}'

In [ ]:
from glob import glob                                                           
pngs = glob('./*.png')

for j in pngs:
    img = cv2.imread(j)
    cv2.imwrite(j[:-3] + 'jpg', img)

In [ ]:
%rm *.png 

### 3d: populate the labels/ directory

In [ ]:
%cp *.txt ../labels/

### 4th: create the auxiliary files

In [ ]:
%cd ../
!touch training_data.txt
!touch validation_data.txt
!touch face_mask_classes.names
!touch face_mask.data

append class names in face_mask_classes.names

In [ ]:
!echo "no face mask" >> face_mask_classes.names
!echo "face mask" >> face_mask_classes.names

configure the data file

In [ ]:
!echo "classes = 2" >> face_mask.data
!echo "train = custom_data/training_data.txt" >> face_mask.data
!echo "valid = custom_data/validation_data.txt" >> face_mask.data
!echo "names = custom_data/face_mask_classes.names" >> face_mask.data
!echo "backup = backup/" >> face_mask.data

In [ ]:
!head face_mask.data

back them up

In [ ]:
%cp face_mask.data ../../drive/MyDrive/my_model/
%cp face_mask_classes.names ../../drive/MyDrive/my_model/

### 5th: split the data into train and validation sets 

and populate the two respective text files with the appropriate file names

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd 
import os 

PATH = 'images/'
list_img=[img for img in os.listdir(PATH) if img.endswith('.jpg')==True]

path_img=[]

for i in range (len(list_img)):
    path_img.append(PATH+list_img[i])
    
df=pd.DataFrame(path_img)

# split 
data_train, data_test, labels_train, labels_test = train_test_split(df[0], df.index, test_size=0.20, random_state=42)

In [ ]:
train_idx=list(data_train.index)
test_idx=list(data_test.index)
    
# relative path to the binary 
relpath = "custom_data/"
backup_path = "/content/drive/MyDrive/my_model/"
# Train file
# Open a file with access mode 'a'
with open("training_data.txt", "a") as file_object:
  for i in range(len(train_idx)):
    file_object.write(relpath+data_train[train_idx[i]]+"\n")   

with open("validation_data.txt", "a") as file_object:
  for i in range(len(test_idx)):
    file_object.write(relpath+data_test[test_idx[i]]+"\n")

back them up

In [ ]:
%cp training_data.txt ../../drive/MyDrive/my_model/
%cp validation_data.txt ../../drive/MyDrive/my_model/

##  Compile DarkNet

In [ ]:
%cd ..

In [ ]:
!make -j4

## Test DarkNet with COCO example data 

In [ ]:
!./darknet detector test cfg/coco.data cfg/yolov4.cfg customization/yolov4.weights data/person.jpg

**confirm findings:**

In [ ]:
from google.colab.patches import cv2_imshow
test_image = cv2.imread("data/person.jpg")
cv2_imshow(test_image)

## Train a Custom Model

### Create a backup directory for the weights

In [ ]:
%rm -r /content/darknet/backup

In [ ]:
%mkdir ../drive/MyDrive/my_model/backup/

In [ ]:
!ln -s /content/drive/MyDrive/my_model/backup/ /content/darknet/

### Train the model

Auxiliary parameters:

- **map** flag: if set, generates a raster plot/graph of the loss and mean average precision
- **dont_show** flag: if set, it prevents attempts to display the progress chart which may cause disruprions in the notebook environment

In [ ]:
!./darknet detector train custom_data/face_mask.data customization/yolov4.cfg customization/yolov4.conv.137 -map -dont_show

It timed out after  9h 17m 45s with approximately 95% of the iterations completed. We used the best weights obtained 

### While Training

- **progress chart**: there is an image file (PNG) generated periodically to report the latest mean average precision (mAP) vs the Loss value for each iteration. It can be found inside the darknet directory. You may download this to check the training progress.
- **log**: there may be times in which it stalls, but that is temporary and the  process is running in the background
- **disconnection/timeout**: in this case check your backup folder in Google drive for which you created the symbolic link. You will find several weight files. Among them there is one generated for every 1000 iterations, one with the best weights computed (highest mAP and not  lowest Loss), and one with  the latest weights before timeout or training  completion. You may chose to reconnect the notebook and start off where you stopped by running the following:

In [ ]:
!./darknet detector train custom_data/face_mask.data customization/yolov4.cfg /content/drive/MyDrive/my_model/backup/yolov4_last.weights -map -dont_show

### Upon Training Completion

Once training is complete, dowload the foloowing:

1. best weights file, i.e. ```MyDrive/my_model/backup/yolov4_best.weights```
2. the configuration file ```/content/darknet/customization/yolov4.cfg```
3. the  summary chart image file ```/content/darknet/chart_yolov4.png```


to deploy the custom yolov4 object dector please see the next notebook titled *object_detection_yolov4_custom_model_deploy.ipynb*
